# Data

In [3]:
import pandas as pd

sampled_data = pd.read_csv('../data/final_train_data.csv', index_col=0)

In [6]:
!mkdir data

In [8]:
sampled_data

,Unnamed: 0,asin,title,price,userId,profileName,reviewHelpfulness,overall,reviewTime,reviewSummary,...,text_lematized,length,sentiment,total_length,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,mapped_sentiment,lengths
0,0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,...,this be only for julie strain fan . it ' s a c...,463,positive,463,0,0,11,0,2,93
1,1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,...,i do not care much for dr. seuss but after rea...,1435,positive,1435,1,0,22,0,2,264
2,2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,...,"if people become the book they read and if "" t...",1763,positive,1763,2,0,28,0,2,321
3,3,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,...,philip nel - dr. seuss : american icon this be...,1552,positive,1552,0,1,27,0,2,286
4,4,0826414346,Dr. Seuss: American Icon,NaN,A2F6NONFUDB6UK,Malvin,2/2,4.0,1127174400,One of America's greatest creative talents,...,""" dr . seuss : american icon "" by philip nel b...",1988,positive,1988,0,0,29,0,2,312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128642,128642,B0007PC4ZM,Collapse: How Societies Choose to Fail or Succeed,NaN,A26ZAM8KFU0GY9,Jim Estill,0/2,4.0,1155081600,How to Thrive,...,i read collapse by jared diamond . this be an ...,524,positive,524,0,0,6,0,2,94
128643,128643,B0007PC4ZM,Collapse: How Societies Choose to Fail or Succeed,NaN,A3KKUNIFVEAHNC,saskatoonguy,7/14,2.0,1143244800,Intriguing bits of world history plus sky-is-f...,...,most of this book consist of story of fail soc...,1709,negative,1709,0,1,20,2,0,293
128644,128644,B0007PC4ZM,Collapse: How Societies Choose to Fail or Succeed,NaN,A1ALF72BQ4RIEL,Bill Pen Name,4/9,5.0,1137024000,A Must Read for all Thoughtful People,...,this work would rate at the top of the scale j...,258,positive,258,0,0,3,0,2,50
128645,128645,B0007PC4ZM,Collapse: How Societies Choose to Fail or Succeed,NaN,A1ZS0YNUQ4UAJ2,Susan Norton,4/9,5.0,1107561600,Full of interest,...,i be not sure if i agree with everything in th...,415,positive,415,0,0,6,0,2,79


In [7]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
for fold, (train_ids, test_ids) in enumerate(cv.split(sampled_data.reviewText, sampled_data['sentiment'])):
    print('======', len(train_ids))
    sampled_data.loc[train_ids].to_csv(f'data/data_train_sampled_for_drill_temp{fold}.csv')
    sampled_data.loc[test_ids].to_csv(f'data/data_test_sampled_for_drill_temp{fold}.csv')
    !export MKL_SERVICE_FORCE_INTEL=1; deepspeed --num_gpus=1 \
           --num_nodes=1 \
           run_transformer_training.py \
           --deepspeed model_config_gpt_contentyze.json \
            --model_name_or_path 'xlnet-base-cased' \
            --train_file data/data_train_sampled_for_drill_temp{fold}.csv \
            --validation_file data/data_test_sampled_for_drill_temp{fold}.csv \
            --do_train \
            --do_eval \
            --report_to wandb \
            --overwrite_cache \
            --output_dir /model_saved/model_trained{fold}/ \
            --num_train_epochs  1  \
            --gradient_accumulation_steps 16  \
            --per_device_train_batch_size 48 \
            --use_fast_tokenizer False      \
            --learning_rate 5e-06 \
            --logging_steps=1 \
            --save_steps=200
    !rm data/data_train_sampled_for_drill_temp{fold}.csv
#             --fp16

====== 102917
[2024-08-15 15:58:32,544] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-08-15 15:58:33,910] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-08-15 15:58:33,927] [INFO] [runner.py:571:main] cmd = /media/veracrypt4/repos/intro-to-supervised-algorithms-in-machine-learning-final-project/venv/bin/python3.10 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None run_transformer_training.py --deepspeed model_config_gpt_contentyze.json --model_name_or_path xlnet-base-cased --train_file data/data_train_sampled_for_drill_temp0.csv --validation_file data/data_test_sampled_for_drill_temp0.csv --do_train --do_eval --report_to wandb --overwrite_cache --output_dir /model_saved/model_trained0/ --num_train_epochs 1 --gradient_accumulation_steps 16 --per_device_train_batch_si

In [24]:
import torch
from amazon_sentiment_analysis_model import AmazonSentimentReviewModel
from transformers import XLNetTokenizer
from datasets import load_dataset
from run_transformer_training import tokenize_function, group_texts
import os
def load_model_from_path(path):
    model = AmazonSentimentReviewModel()
    model_state_dict = torch.load(os.path.join(path, 'pytorch_model.bin'))
    model.load_state_dict(model_state_dict)
    model = model.cuda().eval().half()
    return model

    
def load_dataset_from_path(path):

    datasets = load_dataset('csv', data_files=[path])
    to_remove = [i for i in datasets.column_names['train'] if i not in ['overall', 'mapped_sentiment']]
    column_names = datasets["train"].column_names

    text_column_name = 'reviewText'
    tokenized_datasets = datasets.map(
        lambda x: tokenize_function(x, tokenizer=tokenizer, text_column_name=text_column_name),

        batched=True,
        num_proc=1,
        remove_columns=column_names,
        load_from_cache_file=True,
    )

    lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=1,
        load_from_cache_file=True,
    )
    return lm_datasets




In [12]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")
tokenizer.pad_token = tokenizer.eos_token

In [5]:
torch.cuda.empty_cache()

In [6]:
import gc 
gc.collect()

35

In [27]:
MAPPING = {
    0: "neutral",
    1: "negative",
    2: "positive"
    
}

In [30]:
from sklearn.metrics import f1_score
from operator import truediv
import numpy as np

from glob import glob

files = glob("/model_saved/model_trained*")
text_column_name = "reviewText"
FINAL_F1_SCORES = []


for num, file in enumerate(files):
    model = load_model_from_path(file)
    lm_datasets = load_dataset_from_path(f"data/data_test_sampled_for_drill_temp{num}.csv")
    results = []
    labels = []
    BATCH_SIZE = 256 * 4
    with torch.no_grad():
        for range_ in range(0, len(lm_datasets['train']), BATCH_SIZE):
            x = lm_datasets['train'][range_:range_+BATCH_SIZE]
            inputbatch = torch.tensor(x['input_ids']).to(model.transformer.device)
            token_type_ids_batch = torch.tensor(x['token_type_ids']).to(model.transformer.device)
            attention_mask = torch.tensor(x['attention_mask']).to(model.transformer.device)
            labelbatch = torch.tensor(x['labels']).to(model.transformer.device)
            output = model(inputbatch, attention_mask=attention_mask, token_type_ids=token_type_ids_batch, labels=labelbatch)
            results.extend(torch.argmax(output.logits, dim=-1).detach().cpu().numpy().tolist())
            labels.extend(labelbatch.cpu().detach().numpy().tolist())
            f1 = f1_score([MAPPING[i] for i in labels], [MAPPING[i] for i in results], average='weighted')
            print ('F1 SCORE: {}\n'.format(f1))
    f1 = f1_score([MAPPING[i] for i in labels], [MAPPING[i] for i in results], average='weighted')
    print ('FINAL F1 SCORE: {}\n'.format(f1))
    FINAL_F1_SCORES.append(f1)


        

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

F1 SCORE: 0.7413247778219922

F1 SCORE: 0.7461745585413458

F1 SCORE: 0.748442304558527

F1 SCORE: 0.7545019067898038

F1 SCORE: 0.754092580889097

F1 SCORE: 0.7547631769851101

F1 SCORE: 0.7515443587895044

F1 SCORE: 0.7505385409085439

F1 SCORE: 0.7486612498133688

F1 SCORE: 0.7477341464003349

F1 SCORE: 0.7474489552114598

F1 SCORE: 0.7467809074899913

F1 SCORE: 0.7436606156344383

F1 SCORE: 0.7417933593145448

F1 SCORE: 0.7424124953825643

F1 SCORE: 0.7416219254675522

F1 SCORE: 0.7421186755969726

F1 SCORE: 0.742056758329871

F1 SCORE: 0.742122305234509

F1 SCORE: 0.7410013494523106

F1 SCORE: 0.7411502491020449

F1 SCORE: 0.7416925181778983

F1 SCORE: 0.7418906980631463

F1 SCORE: 0.741721616039735

F1 SCORE: 0.7422644211221808

F1 SCORE: 0.7424716939443242

F1 SCORE: 0.7427348571865556

F1 SCORE: 0.7435031999142379

F1 SCORE: 0.7439089875461701

F1 SCORE: 0.7432703091279717

F1 SCORE: 0.7433735197221749

FINAL F1 SCORE: 0.7433735197221749



Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4ec50749f179ee77/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.



F1 SCORE: 0.7665992633042944

F1 SCORE: 0.7636768719974337

F1 SCORE: 0.7584804590619623

F1 SCORE: 0.7498461266624261

F1 SCORE: 0.7473769722308944

F1 SCORE: 0.7482110938310572

F1 SCORE: 0.7490246500779861

F1 SCORE: 0.7494148656918426

F1 SCORE: 0.7459970183481958

F1 SCORE: 0.74372650818062

F1 SCORE: 0.7431533172420007

F1 SCORE: 0.7411518431018567

F1 SCORE: 0.7412261990368709

F1 SCORE: 0.7421050933969322

F1 SCORE: 0.7415495803558104

F1 SCORE: 0.7410667223399965

F1 SCORE: 0.7412122172323587

F1 SCORE: 0.741187915094213

F1 SCORE: 0.7417760462894067

F1 SCORE: 0.7419942028837747

F1 SCORE: 0.7420175952090722

F1 SCORE: 0.7415520511372852

F1 SCORE: 0.7428923670369996

F1 SCORE: 0.743181007702952

F1 SCORE: 0.7435194094331673

F1 SCORE: 0.7443971877700503

F1 SCORE: 0.7442429761166692

F1 SCORE: 0.7440267649560776

F1 SCORE: 0.743527161322397

F1 SCORE: 0.7435026028225441

F1 SCORE: 0.7437152626590623

FINAL F1 SCORE: 0.7437152626590623



...

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-41d63c500ab7af5c/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.



F1 SCORE: 0.7437192627514955

F1 SCORE: 0.7470739351658362

F1 SCORE: 0.7398603957153606

F1 SCORE: 0.742592551885368

F1 SCORE: 0.743877489320211

F1 SCORE: 0.7455653387478677

F1 SCORE: 0.748119356079009

F1 SCORE: 0.7465861364817309

F1 SCORE: 0.7431873696112071

F1 SCORE: 0.7444439712500568

F1 SCORE: 0.7458345566090745

F1 SCORE: 0.7461574182818822

F1 SCORE: 0.7462475859043628

F1 SCORE: 0.7474517624950613

F1 SCORE: 0.7482391221147097

F1 SCORE: 0.7493891841771696

F1 SCORE: 0.7498511560451078

F1 SCORE: 0.7504074777702798

F1 SCORE: 0.7509660942418178

F1 SCORE: 0.7506832195504619

F1 SCORE: 0.7506263176184065

F1 SCORE: 0.7499312996136446

F1 SCORE: 0.7492229229109683

F1 SCORE: 0.7490248840695335

F1 SCORE: 0.7488515561527018

F1 SCORE: 0.7493262635042469

F1 SCORE: 0.7486715708778525

F1 SCORE: 0.7485759626861933

F1 SCORE: 0.7482115122561186

F1 SCORE: 0.7487667055316762

F1 SCORE: 0.7487299423522583

FINAL F1 SCORE: 0.7487299423522583



Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-833d3dd3e2b6e9e0/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.



F1 SCORE: 0.7479730526787047

F1 SCORE: 0.7507885484704045

F1 SCORE: 0.7513563853184965

F1 SCORE: 0.7521428149061142

F1 SCORE: 0.7549975270491388

F1 SCORE: 0.7540600440765117

F1 SCORE: 0.7549126221112671

F1 SCORE: 0.7578435209055241

F1 SCORE: 0.7590612941815089

F1 SCORE: 0.7571638494253484

F1 SCORE: 0.7554914175686633

F1 SCORE: 0.7546620854848146

F1 SCORE: 0.7544439219028498

F1 SCORE: 0.7544474180729873

F1 SCORE: 0.7542761223504189

F1 SCORE: 0.7537798053480032

F1 SCORE: 0.7542377915263115

F1 SCORE: 0.7526311540701737

F1 SCORE: 0.7523603655601825

F1 SCORE: 0.7517848321494629

F1 SCORE: 0.7532204110536727

F1 SCORE: 0.7533294751953398

F1 SCORE: 0.7518165975746587

F1 SCORE: 0.7518952232485243

F1 SCORE: 0.751996746078432

F1 SCORE: 0.7524445085666767

F1 SCORE: 0.752356705458871

F1 SCORE: 0.7519716062350695

F1 SCORE: 0.7513650177141584

F1 SCORE: 0.7516984448768286

F1 SCORE: 0.7518877864062338

FINAL F1 SCORE: 0.7518877864062338



In [32]:
import pickle
with open('roberta_scores.pkl', 'wb') as f:
    pickle.dump(FINAL_F1_SCORES, f)